In [3]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import csv

In [4]:
df = pd.read_csv('shashadenim.csv')
df

Date  Price   Open   High    Low     Vol. Change %
0     Jan 09, 2020  23.10  23.70  23.90  23.00   37.41K   -0.43%
1     Jan 08, 2020  23.20  23.70  23.90  23.10   48.49K   -2.11%
2     Jan 07, 2020  23.70  23.90  24.20  23.60   96.75K   -0.84%
3     Jan 06, 2020  23.90  23.90  24.80  23.80   86.34K   -0.83%
4     Jan 05, 2020  24.10  24.90  24.90  23.90  114.78K   -2.43%
...            ...    ...    ...    ...    ...      ...      ...
1163  Mar 12, 2015  28.26  28.70  29.13  27.97    4.97M   -1.26%
1164  Mar 11, 2015  28.62  27.31  29.21  27.09    8.48M    5.38%
1165  Mar 10, 2015  27.16  27.89  28.40  27.02    3.82M   -3.14%
1166  Mar 09, 2015  28.04  27.24  28.91  27.24    5.47M    3.24%
1167  Mar 08, 2015  27.16  27.02  27.75  26.43    4.97M    0.26%

[1168 rows x 7 columns]

In [5]:
df.shape

(1168, 7)

In [41]:
plt.figure(figsize=(16,8))
plt.title('SHASHADENIM Close Price History')
plt.plot(df['Price'], linewidth = 1)
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price (BDT)', fontsize=18)
plt.show()

In [22]:
# create dataframe only with close column
data = df.filter(['Price'])
# dataframe to numpy array
dataset = data.values
training_data_len = math.ceil(len(dataset) * 0.8)
training_data_len

935

In [23]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(dataset)
scaled_data

array([[0.        ],
       [0.00210438],
       [0.01262626],
       ...,
       [0.08543771],
       [0.10395623],
       [0.08543771]])

In [24]:
train_data = scaled_data[0:training_data_len, :]
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    if i <= 61:
        print(x_train)
        print(y_train)
        print()


[array([0.        , 0.00210438, 0.01262626, 0.01683502, 0.02104377,
       0.03367003, 0.02946128, 0.03367003, 0.03156566, 0.0273569 ,
       0.02104377, 0.01683502, 0.02946128, 0.0273569 , 0.01683502,
       0.02525253, 0.03998316, 0.05260943, 0.03156566, 0.03577441,
       0.04419192, 0.0547138 , 0.0736532 , 0.09048822, 0.09048822,
       0.08838384, 0.07786195, 0.08417508, 0.08207071, 0.0736532 ,
       0.03998316, 0.01893939, 0.00883838, 0.0128367 , 0.01094276,
       0.00484007, 0.00883838, 0.02504209, 0.04692761, 0.05702862,
       0.06502525, 0.06902357, 0.05892256, 0.06502525, 0.0831229 ,
       0.0770202 , 0.08501684, 0.04503367, 0.09701178, 0.11510943,
       0.09301347, 0.08101852, 0.08101852, 0.09701178, 0.11321549,
       0.1172138 , 0.1172138 , 0.13110269, 0.08501684, 0.09701178])]
[0.1390993265993266]

[array([0.        , 0.00210438, 0.01262626, 0.01683502, 0.02104377,
       0.03367003, 0.02946128, 0.03367003, 0.03156566, 0.0273569 ,
       0.02104377, 0.01683502, 0.029

In [25]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [26]:
# Reshape the data
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

(875, 60, 1)

In [27]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))


In [28]:
#compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [29]:
#train the model
model.fit(x_train, y_train, batch_size=1, epochs=1)

Epoch 1/1
875/875 [==============================] - 32s 36ms/step - loss: 0.0059


In [30]:
#create a new array containing scaled values from 1543 to 2003
test_data = scaled_data[training_data_len-60: , :]
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])
    


In [31]:
x_test = np.array(x_test)
type(x_test)

numpy.ndarray

In [32]:
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
x_test.shape

(233, 60, 1)

In [33]:
# get the models predicted price values
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
len(predictions)

233

In [34]:
# get the root mean squared error (RMSE)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
rmse

0.7711432942402721

In [35]:
# plot the data
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions
valid

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Price  Predictions
935   34.68    33.497383
936   34.85    33.596127
937   35.18    33.695503
938   34.34    33.824799
939   34.01    33.842644
...     ...          ...
1163  28.26    26.639128
1164  28.62    26.695322
1165  27.16    26.872969
1166  28.04    26.915453
1167  27.16    26.987957

[233 rows x 2 columns]

In [50]:
# visualize the data
plt.figure(figsize=(16,8))
plt.title('SHASHADENIM Stock Prediction')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price (BDT)', fontsize=18)
plt.plot(train['Price'], linewidth = 1)
plt.plot(valid, linewidth=2)
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()

In [44]:
valid

Price  Predictions
935   34.68    33.497383
936   34.85    33.596127
937   35.18    33.695503
938   34.34    33.824799
939   34.01    33.842644
...     ...          ...
1163  28.26    26.639128
1164  28.62    26.695322
1165  27.16    26.872969
1166  28.04    26.915453
1167  27.16    26.987957

[233 rows x 2 columns]